# Project 3: Measuring the correlation between data content and filenames

#### Author: **Thomas Casey**
##### Location: University of California Santa Barbara

## Table of Contents
1. [Business Understanding](#Business-Understanding)
2. [Data Understanding](#Data-Understanding)
3. [Data Preparation](#Data-Preparations)
4. [Modeling](#Method-Description)
5. [Evaluation](#Evaluation)
6. [Deployment](#Deployment)

<a id="Business Understanding"> </a>
## Business Understanding

Can undocumented and unstructured data be structured and interpreted with an acceptable level of confidence based only on the raw content of the data and filenames associated with the data?


<a id="Data Understanding"> </a>
## Data Understanding

The data are generated using an experiment called Overhauser Dynamic Nuclear Polarization (ODNP). The data should exist in blocks, meaning certain sizeed collections of files and folders consititute one piece of data. The routine that collects and categorizes the data is known and some aspects of the structure of the data are already understood:
1. Each "sample" corresponds to a base folder that contains folders named "1" through "33", "304", "503, "700", "701", and also two .mat files OR two .csv files; one named "power" and the other named "t1_powers". The folders can either be of type null or contain additional files that yield a one dimensional spectrum or two dimensional spectra in a proprietary format. The .mat or .csv files contain lists of continuous measurements of microwave power level made over the span of approximately two hours. 
2. One of the one dimensional spectra is considered an "off" spectrum and several others are to be compared to it once it is identified. Most of the spectra are one dimensional and should present a smooth trend in changing intensities if organized properly.
3. A subset are two dimensional spectra and should also present a smooth trend in changing intensities. 
4. The measurements in the .mat and .csv files should correspond in time to the time stamp of certain files in each numbered folder. These arrays should be condensed to a length that matches the total number of spectra by sectioning based on time intervals and averaging each section individually. 
5. The one dimensional spectra must be processed and condensed to a single amplitude that is the integral of the spectrum. The two dimensional spectra must be condensed to a single number "T1" that is the result of processing, integrating, and fitting the trend in integrals within the 2D set to an exponential function.
6. When properly arranged, the one dimensional data should yield a curve of spectral amplitudes that increases asmyptotically and the two dimensional data another curve of "T1" values that increases linearly. The theory behind and form of the analytical models for fitting the data can be found in [Prog. Nuc. Mag. Res.](http://dx.doi.org/10.1016/j.pnmrs.2013.06.001) and [Methods in Enzymology](https://doi.org/10.1016/bs.mie.2018.09.024).  

<a id="Data Preparation"> </a>
## Data Preparation

For handling this data most efficiently I use a python package that I helped develop, and currently maintain, called [DNPLab](DNPLab.net). This package contains functions for loading proprietary data formats, processing data, and modeling the data using analytical functions. First it is helpful to take apart one sample folder to learn how to handle the rest of the data,

In [ ]:
import dnplab                      # import the DNPLab package
from dnplab.dnpImport import load  # condense the syntax for loading data
import os                          # import os for using path tools
import numpy as np                 # import numpy for useful tools
import matplotlib.pyplot as plt    # use matplotlib to create some plots for visualizing the data

base_path = "../test_set"
paths = os.listdir(base_path)      # create a list of paths to the folders inside the test folder. Each
                                   # should refer to one spectrum if it is actual data
flag = []
for indx, path in enumerate(       # loop through paths attempting to interpret them as data
    paths
):  
    try:
        data = load(os.path.join(base_path, path))
        flag.append("DATA")        # successful loading is marked as DATA
    except:
        flag.append("NULL")        # errors marked as NULL
        continue

truth_table = np.column_stack(     # construct a 2D list where each folder path is DATA or NULL
    (paths, flag)
)  

Visualizing some of the data shows one dimensional spectra are in the form of free-induction decays,

In [ ]:
data = load(os.path.join(base_path, paths[10]))
plt.plot(data.values)
plt.show()

Two dimensional data are as well,

In [ ]:
data = load(os.path.join(base_path, paths[1]))
plt.plot(data.values)
plt.show()

Now that the relevant data are identified, it must be arrange for modeling. I start by condensing to just the usable data from the collection of good and NULL data. I'll store each spectrum to a dictionary where the folder numbers are the keys,

In [ ]:
data_dict = {}
for indx, path in enumerate(paths):
    print(indx)
    if truth_table[indx, 1] == "DATA":
        data_dict[truth_table[indx, 0]] = load(os.path.join(base_path, path))
    else:
        pass

Finally, extract and isolate the target characterisitcs of the data using built in DNPLab functions,

In [ ]:
final_target_1D = []
folders_1D = []
final_target_2D = []
folders_2D = []
data_dimensions = []
for indx, spec in enumerate(data_dict.keys()):
    workspace = dnplab.create_workspace("proc", data_dict[spec])
    dnplab.dnpNMR.remove_offset(workspace)
    dnplab.dnpNMR.window(workspace, linewidth=10)
    dnplab.dnpNMR.fourier_transform(workspace, zero_fill_factor=2)
    dnplab.dnpNMR.autophase(workspace, force_positive=False)
    data_dimensions.append(workspace["proc"].ndim)
    dnplab.dnpTools.integrate(workspace)
    if data_dimensions[indx] == 1:
        final_target_1D.append(workspace["proc"].values)
        folders_1D.append(spec)
    elif data_dimensions[indx] == 2:
        dnplab.dnpFit.exponential_fit(workspace, type="T1")
        final_target_2D.append(workspace["fit"].attrs["T1"])
        folders_2D.append(spec)
        

folders_1D = list(map(int, folders_1D))
folders_1D.sort()
folders_2D = list(map(str, folders_2D))
folders_2D.sort()

powers_1D = dnplab.dnpIO.cnsi.get_powers(base_path, "power", folders_1D)
powers_2D = dnplab.dnpIO.cnsi.get_powers(base_path, "t1_powers", folders_2D)

The data are now arranged into the target format; a list of amplitudes of the one dimensional spectra and another list of "T1" values for the two dimensional data. Each list has a corresponding power list.

<a id="Modeling"> </a>
## Modeling

Models exist for fitting data of this nature and have been programmed into a module called "dnpHydration" within the DNPLab package. The data, along with some known parameters in the form of a dictionary, are passed to the function and physical constants are calculated using optimization routines,

In [ ]:
hydration = {
    "E": np.array(final_target_1D),
    "E_power": np.array(powers_1D),
    "T1": np.array(final_target_2D),
    "T1_power": np.array(powers_2D),
}
hydration.update(
    {
        "T10": T10,
        "T100": T100,
        "spin_C": spin_C,
        "field": field,
        "smax_model": smax_model,
        "t1_interp_method": t1_interp_method,
    }
)
hyd = dnplab.create_workspace()
hyd.add("hydration_inputs", hydration)

results = dnplab.dnpHydration.hydration(hyd)

print(results)

<a id="Evaluation"> </a>
## Evaluation

A small set of known data that were previously manually analyzed and published was passed through this routine to confirm the model produces the correct published result.

<a id="Deployment"> </a>
## Deployment

An example application was the batch processing of a large dataset for which each of the base folders had clues in their name but there was no available description of the dataset. I processed the entire batch using the procedure above and tried to correlate the clues in the folder names with the results. First I placed the above code into a function to more easily perform the above procedure on an entire batch at once, 

In [ ]:
def process(base_path):
 
    # insert above code

    return results["hydration_results"]["k_sigma"]

Next, I loop over the entire set of base folders creating a dictionary that can be arranged into a simple table,

In [ ]:
base_list = os.listdir(set_path)

descriptive = []
for indx, path in enumerate(base_list):
    descriptive.append(process(path))
    

Next, it is useful to take apart the folder names and give the pieces their own column in the table for more easily making correlations with the descriptive parameter. 

In [ ]:
folder_list = os.listdir(root_path)  # now get the names of all base folders in the root folder

date = []
sample = []
time = []
index = []
for indx, name in enumerate(folder_list):
    nm = name.split("_")
    if "samp6" in nm[2]:
        date.append(nm[0])
        sample.append(nm[2])
        time.append(nm[3])
        index.append(nm[4])


descriptive_dict = {"date": date,
                 "sample": sample,
                 "time": time,
                 "index": index,
                 "descriptor": descriptive,
                 }



In [1]:
date = []
sample = []
time = []
index = []
for indx, name in enumerate(folder_list[folder_list.columns[0]]):
    nm = name.split("_")
    if "samp6" in nm[2]:
        date.append(nm[0])
        sample.append(nm[2])
        time.append(nm[3])
        index.append(nm[4])


descriptive_dict = {"date": date,
                 "sample": sample,
                 "time": time,
                 "index": index,
                 "descriptor": descriptive,
                 }

Now package the data and create a database to store and interact with the data in table format using SQL. First, initial the database, 

In [ ]:
# I'll use IBM cloud to host the database
import ibm_db

dsn_hostname = ""
dsn_uid = ""      
dsn_pwd = ""      

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"       
dsn_port = "50000"     
dsn_protocol = "TCPIP"  

dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

conn = ibm_db.connect(dsn, "", "")
print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)


Next, create a table,

In [2]:
make_table = "CREATE TABLE ODNP(ID INTEGER PRIMARY KEY NOT NULL, DATE VARCHAR(20), SAMPLE CHAR(5), TIME CHAR(2), INDEX CHAR(2), DESCRIPT VARCHAR(20))"

initiate_table = ibm_db.exec_immediate(conn, make_table)

NameError: name 'plt' is not defined

The packages pandas and seaborn are useful for handling and visualizing the data. So I use these packages to upload the data to the database and also retireve and analyze,

In [3]:
import pandas as pd
from sqlalchemy import create_engine
db_eng = create_engine('sqlite://', echo=False)

db_frame = pd.dataframe(descriptive_dict) 

db_frame.to_sql('ODNP', con=db_eng)

NameError: name 'plt' is not defined

Now retrieve the data and make some plots plot,

In [ ]:
import ibm_db_dbi

connect = ibm_db_dbi.Connection(conn)

sel_stat = "select * from INSTRUCTOR"

q_dataframe = pandas.read_sql(sel_stat, connect)

sample_column = q_dataframe.SAMPLE
descriptive_column = q_dataframe.DESCRIPT

import seaborn as sns
import matplotlib.pyplot as plt
